## Создание корпуса и разметка

Для морфологической разметки я собрала небольшой корпус из 230 словоупотреблений на основе предложений из НКРЯ. После чего я сделала его частеречную разметку, предварительно создав csv-файл с токенами и номерами предложений, к которым они относятся

In [1]:
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import csv
import pandas as pd
from collections import Counter

In [2]:
with open('minicorpus.txt', 'r', encoding='utf-8') as txt_file:
     lines = txt_file.readlines()
extra = '―«»'
tokens = []
for line in lines:
    line_tokens = word_tokenize(line)
    line_tokens = [token for token in line_tokens 
                   if token[0] not in punctuation and token[0] not in extra and not token.isnumeric()]
    line_tokens[0] = line_tokens[0].lower()
    tokens.append(line_tokens)

In [3]:
tokens[:2]

[['нежданный',
  'снег',
  'шуршал',
  'всю',
  'ночь',
  'под',
  'утро',
  'стих',
  'и',
  'тут',
  'же',
  'его',
  'влажную',
  'шкуру',
  'схватило',
  'морозцем'],
 ['ты',
  'вечности',
  'заложник',
  'у',
  'времени',
  'в',
  'плену',
  'сколько',
  'раз',
  'вспоминая',
  'этот',
  'стих',
  'я',
  'сбивался',
  'забывал',
  'где',
  'тут',
  'время',
  'где',
  'вечность']]

In [4]:
# with open('tokens.csv', 'w', encoding='utf-8') as csv_file:
#     csvwriter = csv.writer(csv_file)
#     csvwriter.writerow(['token', 'sentence', 'pos'])
#     for i in range(len(tokens)):
#         for token in tokens[i]:
#             csvwriter.writerow([token, i+1, None])

Приведу примеры предложений, которые попали в корпус:  
_(1) Нежданный снег шуршал всю ночь, под утро стих, и тут же его влажную шкуру схватило морозцем.  
(2) "Ты ― вечности заложник, у времени в плену" ― сколько раз, вспоминая этот стих, я сбивался, забывал, где тут время, где вечность.  
(3) В принципе, можно было бы продолжать и дальше, но, думается, это тот случай, когда и так все ясно.  
(4) Американская гостья из Риги, возможно, одна из немногих иностранных лидеров, кто ясно представляет себе, когда «начнётся развитие событий вокруг Ирака»._  
  
В первых двух предложениях мы видим омоним "стих", который в __предложении 1__ является глаголом, а в __предложении 2__ существительным. И в __предложениях 3 и 4__ слово "ясно" представлено разными частями речи. В одном оно является прилагательном, а в другом — наречием.

Мой тегсет выглядит так: NOUN, VERB, ADJ, ADV, PREP, CONJ, PART, PRED  
Я выбрала такой набор тегов, потому что он, как мне кажется, наиболее удобен при расхождениях в определении и обозначении частей речи таких слов как _весь_, _вспоминая_, _Анна_. Перечисленные слова я размечала как ADJ, VERB и NOUN соотвественно, не уточняя, что первое — местоименное прилагательное, второе — деепричастие, а третье — имя собственное.

## Парсеры

Дальше я "прогнала" свой корпус через три парсера: __mystem__, __pymorphy__ и __spacy__

In [5]:
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import spacy

### Spacy

In [6]:
nlp = spacy.load("ru_core_news_sm")
with open('spacy.csv', 'w', encoding='utf-8') as csv_file:
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(['token', 'sentence', 'pos'])
    for n in range(len(lines)):
        sent = nlp(lines[n])
        for i in range(len(sent)):
            token = sent[i].text
            if sent[i].pos_ not in ['PUNCT', 'SPACE'] and not token.isnumeric():
                if i == 0:
                    token = sent[i].text.lower()
                csvwriter.writerow([token, n+1, sent[i].pos_])

### Mystem

In [7]:
m = Mystem()

In [8]:
with open('mystem1.csv', 'w', encoding='utf-8') as csv_file:
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(['token', 'sentence', 'pos'])
    for n in range(len(lines)):
        ana = m.analyze(lines[n])
        for i in range(len(ana)):
            if 'analysis' in ana[i]:
                token = ana[i]['text']
                if i == 0:
                    token = token.lower()
                gr = ana[i]['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                csvwriter.writerow([token, n+1, pos])

### Pymorphy

In [9]:
morph = MorphAnalyzer()

In [10]:
with open('pymorphy1.csv', 'w', encoding='utf-8') as csv_file:
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(['token', 'sentence', 'pos'])
    for n in range(len(tokens)):
        for token in tokens[n]:
            first = morph.parse(token)[0]  # будем выбирать первый разбор токена
            pos = first.tag.POS
            csvwriter.writerow([token, n+1, pos])

## Оценим accuracy теггеров

In [11]:
def standardize(csvfile):
    df = pd.read_csv(csvfile)
    df['pos'] = df['pos'].replace(['DET', 'ADJF', 'ADJS', 'A', 'COMP', 'APRO'], 'ADJ')
    df['pos'] = df['pos'].replace(['INFN', 'PRTF', 'PRTS', 'GRND', 'V', 'AUX'], 'VERB')
    df['pos'] = df['pos'].replace(['CCONJ', 'SCONJ'], 'CONJ')
    df['pos'] = df['pos'].replace(['ADP', 'PR'], 'PREP')
    df['pos'] = df['pos'].replace(['NPRO', 'SPRO'], 'PRON')
    df['pos'] = df['pos'].replace(['ADVB', 'ADVPRO'], 'ADV')
    df['pos'] = df['pos'].replace(['PROPN', 'S'], 'NOUN')
    df['pos'] = df['pos'].replace('PRTCL', 'PART')
    df.to_csv(csvfile, index=False)

In [12]:
standardize('spacy.csv')
standardize('mystem.csv')
standardize('pymorphy.csv')

In [13]:
my_df = pd.read_csv('tokens.csv')
y = list(my_df['pos'])

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
def count_accuracy(y, parser_file):
    df = pd.read_csv(parser_file)
    y_pred = list(df['pos'])
    accuracy = accuracy_score(y, y_pred)
    return accuracy

In [16]:
spacy_acc = count_accuracy(y, 'spacy.csv')
mystem_acc = count_accuracy(y, 'mystem.csv')
pymorphy_acc = count_accuracy(y, 'pymorphy.csv')
print(f'У Spacy accuracy={spacy_acc}')
print(f'У Mystem accuracy={mystem_acc}')
print(f'У Pymorphy accuracy={pymorphy_acc}')

У Spacy accuracy=0.9118942731277533
У Mystem accuracy=0.960352422907489
У Pymorphy accuracy=0.8722466960352423


## Chunker

In [17]:
def pos(word):
    pos = None
    if 'analysis' in word:
        token = word['text']
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
    return pos

In [18]:
def old_chunker(lines):
    bigrams = []
    for n in range(len(lines)):
        ana = m.analyze(lines[n])
        for i in range(2, len(ana)):
            pos1 = pos(ana[i])
            if pos1 == 'S':
                pos2 = pos(ana[i-2])
                if pos2 == 'A' or pos2 == 'V':
                    phrase = ana[i-2]['text'] + ' ' + ana[i]['text']
                    bigrams.append(phrase.lower())
            elif pos1 == 'V' and ana[i-2]['text'] == 'не':
                phrase = 'не ' + ana[i]['text']
                bigrams.append(phrase.lower())
    return bigrams

In [19]:
old_chunker(lines)

['нежданный снег',
 'влажную шкуру',
 'схватило морозцем',
 'вегетарианская столовая',
 'успокаивающий настой',
 'столовая ложка',
 'общественной жизни',
 'бывалый служака',
 'не нарушай',
 'нарушай дистанции',
 'тупые смски',
 'показного мнения',
 'интересные данные',
 'последние годы',
 'чёрных дыр',
 'звёздной массы',
 'не выполнять',
 'данные обещания',
 'тёплого дома',
 'холодное утро',
 'не хочет',
 'ловить рыбу',
 'американская гостья',
 'иностранных лидеров',
 'начнётся развитие']

## Применение чанкера
Теперь применим наш чанкер к программе из дз 1  
_Примечание_. При использовании в программе определения тональности отзыва я его отредактировала, чтобы он не "выплевывал" словосочетания, а заменял их последовательность слов с нижним подчеркиванием типа _целый_день_

In [20]:
def pos(word):
    pos = None
    if 'analysis' in word and word['analysis']:
        token = word['text']
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
    return pos

In [21]:
def chunker(lines):
    for n in range(len(lines)):
        ana = m.analyze(lines[n])
        for i in range(2, len(ana)):
            pos1 = pos(ana[i])
            if pos1 == 'S':
                pos2 = pos(ana[i-2])
                if pos2 == 'A' or pos2 == 'V':
                    old = ana[i-2]['text'] + ' ' + ana[i]['text']
                    new = ana[i-2]['text'] + '_' + ana[i]['text']
                    lines[n] = lines[n].replace(old, new.lower())
            elif pos1 == 'V' and ana[i-2]['text'] == 'не':
                old = 'не ' + ana[i]['text']
                new = 'не_' + ana[i]['text']
                lines[n] = lines[n].replace(old, new.lower())
    return lines

In [22]:
df = pd.read_csv('data.csv')
df.head()

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5


In [23]:
df.dropna(inplace=True)
df = df[df['Rating'] <= 5]

In [24]:
neg_reviews = list(df['Review'][df['Rating'] < 4][:55])
neg_train = neg_reviews[:50]
neg_test = neg_reviews[50:]

pos_reviews = list(df['Review'][df['Rating'] >= 4][:55])
pos_train = pos_reviews[:50]
pos_test = pos_reviews[50:]

X_train = pos_train + neg_train
X_test = pos_test + neg_test
y_train = [1] * 50 + [0] * 50
y_test = [1] * 5 + [0] * 5

In [25]:
morph = MorphAnalyzer()

In [26]:
sw = stopwords.words('russian')

def lemmatize(reviews_list):
    lemmas_w_noise = [morph.parse(word)[0].normal_form for review in reviews_list
                      for sent in nltk.sent_tokenize(review)
                      for word in nltk.word_tokenize(sent)]
    text = ' '.join(lemmas_w_noise)
    lines = nltk.sent_tokenize(text)
    sentences = chunker(lines)
    lemmas = [lemma for sent in sentences for lemma in nltk.word_tokenize(sent)
              if lemma not in punctuation and lemma not in sw]
    return lemmas

In [27]:
def most_common(lemmas):
    counted = Counter(lemmas)
    new_lemmas = set({k:v for k,v in counted.items() if v > 2 })
    return new_lemmas

In [28]:
neg_lemmas = lemmatize(neg_train)
neg_lemmas = most_common(neg_lemmas)

pos_lemmas = lemmatize(pos_train)
pos_lemmas = most_common(pos_lemmas)

In [29]:
pos_lemmas.difference_update(neg_lemmas)
neg_lemmas.difference_update(pos_lemmas)

In [30]:
def lemmatize_text(review):
    lemmas_w_noise = [morph.parse(word)[0].normal_form 
                      for sent in nltk.sent_tokenize(review)
                      for word in nltk.word_tokenize(sent)]
    lemmas = [lemma for lemma in lemmas_w_noise
              if lemma not in punctuation and lemma not in sw]
    return lemmas

In [31]:
def review_type(text):
    lemmas = set(lemmatize_text(text))
    if len(lemmas.intersection(pos_lemmas)) >= len(lemmas.intersection(neg_lemmas)):
        return 1
    else:
        return 0

In [32]:
y_pred_tr = [review_type(r) for r in X_train]
y_pred_te = [review_type(r) for r in X_test]

In [33]:
train_acc = accuracy_score(y_train, y_pred_tr)
test_acc = accuracy_score(y_test, y_pred_te)

In [34]:
print(train_acc)
print(test_acc)

0.55
0.3


Таким образом, мы видим, что наш чанкер улучшил результат только на трейне и то на 0.01, а на тестовых данных качество не изменилось. Я думаю, так вышло потому, что данных очень мало